In [1]:
import pandas as pd

# Load live and projected rankings
live = pd.read_csv("data/rankings_LIVE_L3.csv")
proj = pd.read_csv("data/rankings_L3.csv")

print("✅ Data loaded")
display(live.head(3))
display(proj.head(3))

✅ Data loaded


,INDEX,PLAYER_NORM,G,MPG,PTS,REB,AST,STL,BLK,3PM,FG%,FT%,TO,RANK_SCORE,RANK
0,10003.0,victor wembanyama,6.0,33.8,26.7,13.7,3.2,1.3,4.7,1.0,0.527,0.792,2.5,15.775352,1
1,10001.0,nikola jokic,5.0,34.8,20.4,14.4,10.8,1.8,0.8,1.0,0.591,0.864,3.0,12.042041,2
2,10014.0,tyrese maxey,6.0,42.5,33.7,4.8,9.0,1.2,0.8,4.3,0.463,0.877,2.3,11.639134,3


,INDEX,PLAYER_NORM,G,MPG,PTS,REB,AST,STL,BLK,3PM,FG%,FT%,TO,RANK_SCORE,RANK
0,10003.0,victor wembanyama,67.8,32.961765,25.945455,11.370455,4.011364,1.227273,3.763636,2.261364,0.500750,0.818023,3.506818,14.917878,1
1,10001.0,nikola jokic,73.2,35.717647,28.077273,12.304545,10.011364,1.627273,0.734091,1.856818,0.578182,0.807795,3.193182,13.427964,2
2,10004.0,luka doncic,70.8,35.841176,30.472727,8.425000,8.588636,1.640909,0.488636,3.595455,0.475841,0.782977,3.765909,11.207812,3


In [2]:
# Define your roster by INDEX numbers
# Example: roster = [101, 204, 305]
roster = [
10001, #Nikola Jokic
10003, #Victor Wembanyama
10004, #Luka Doncic
10010, #James Harden
10017, #LaMelo Ball
10024, #Amen Thompson
10030, #Chet Holmgren
10033, #Josh Giddey
]

print(f"Roster contains {len(roster)} players")

Roster contains 8 players


In [3]:
# Cell 3 — Display Roster (robust to column names)

import numpy as np

# 1) Create LIVE aliases if needed
live_view = live.copy()
if "RANK_SCORE_LIVE" not in live_view.columns and "RANK_SCORE" in live_view.columns:
    live_view["RANK_SCORE_LIVE"] = live_view["RANK_SCORE"]
if "RANK" not in live_view.columns and "RANK_LIVE" in live_view.columns:
    live_view["RANK"] = live_view["RANK_LIVE"]

# 2) Merge LIVE with only projected rank score
proj_view = proj[["INDEX", "PLAYER_NORM", "RANK_SCORE"]].rename(
    columns={"RANK_SCORE": "RANK_SCORE_PROJ"}
)
merged = live_view.merge(proj_view, on=["INDEX", "PLAYER_NORM"], how="left")

# 3) Filter to roster
roster_df = merged[merged["INDEX"].isin(roster)].copy()

# 4) Sort by live rank score
if "RANK_SCORE_LIVE" in roster_df.columns:
    roster_df = roster_df.sort_values("RANK_SCORE_LIVE", ascending=False).reset_index(drop=True)
else:
    print("⚠️ RANK_SCORE_LIVE not found after aliasing. Check your CSV headers.")
    display(roster_df.head())
    
# 5) Reorder columns (keep only those that exist)
desired_cols = [
    "INDEX", "PLAYER_NORM", "G", "MPG", "PTS", "REB", "AST", "STL", "BLK",
    "3PM", "FG%", "FT%", "TO", "RANK_SCORE_PROJ", "RANK_SCORE_LIVE", "RANK"
]
final_cols = [c for c in desired_cols if c in roster_df.columns]
roster_df = roster_df[final_cols]

In [4]:
# 🏀 Cell 4 — Starting Lineup Summary

# 1) Define your starting lineup
starting_lineup = [
10001, #Nikola Jokic
10003, #Victor Wembanyama
10004, #Luka Doncic
10010, #James Harden
10017, #LaMelo Ball
10024, #Amen Thompson
10030, #Chet Holmgren
10033, #Josh Giddey
]

In [5]:

# 2) Filter to starting lineup
starters_df = roster_df[roster_df["INDEX"].isin(starting_lineup)].copy()

if starters_df.empty:
    print("⚠️ No matches found for your starting lineup indexes.")
    starters_summary_df = starters_df.copy()
else:
    # 3) Columns to multiply (numeric, not percentages, and not excluded)
    exclude_cols = ["INDEX", "G", "RANK_SCORE_PROJ", "RANK_SCORE_LIVE", "RANK"]
    pct_cols = [c for c in starters_df.columns if c.endswith("%")]
    multiply_cols = [
        c for c in starters_df.columns
        if starters_df[c].dtype in ["int64", "float64"]
        and c not in exclude_cols
        and c not in pct_cols
    ]

    # 4) Apply 3.5x multiplier to counting stats
    starters_df[multiply_cols] = starters_df[multiply_cols] * 3.5

    # 5) Totals and averages
    total_row = starters_df[multiply_cols].sum(numeric_only=True)
    pct_avgs = starters_df[pct_cols].mean(numeric_only=True) if pct_cols else pd.Series(dtype=float)

    # 6) Build summary row (use NaN so numeric formatting still applies later)
    summary = pd.concat([total_row, pct_avgs])
    summary["PLAYER_NORM"] = "TOTAL / AVG"
    summary["INDEX"] = np.nan
    for c in ["G", "RANK_SCORE_PROJ", "RANK_SCORE_LIVE", "RANK"]:
        if c in starters_df.columns:
            summary[c] = np.nan

    # 7) Append summary and match roster_df column order
    starters_summary_df = pd.concat([starters_df, pd.DataFrame([summary])], ignore_index=True)
    starters_summary_df = starters_summary_df[[c for c in roster_df.columns if c in starters_summary_df.columns]]

In [6]:
# 🧩 Cell 5 — Pretty display (2 decimals, blank NaN, includes INDEX & rank scores)

def pretty2(df):
    df2 = df.copy()

    # Ensure key numeric columns stay numeric for formatting
    for c in ["INDEX", "RANK_SCORE_PROJ", "RANK_SCORE_LIVE"]:
        if c in df2.columns:
            df2[c] = pd.to_numeric(df2[c], errors="coerce")

    # Build formatter for all numeric columns
    num_cols = df2.select_dtypes(include="number").columns.tolist()
    fmt = {c: "{:,.2f}".format for c in num_cols}

    return df2.style.format(fmt, na_rep="")

print("📋 Roster view:")
display(pretty2(roster_df))

print("\n🏀 Starters view (with TOTAL/AVG):")
display(pretty2(starters_summary_df))

📋 Roster view:


,INDEX,PLAYER_NORM,G,MPG,PTS,REB,AST,STL,BLK,3PM,FG%,FT%,TO,RANK_SCORE_PROJ,RANK_SCORE_LIVE,RANK
0,"10,003.00",victor wembanyama,6.00,33.80,26.70,13.70,3.20,1.30,4.70,1.00,0.53,0.79,2.50,14.92,15.78,1.00
1,"10,001.00",nikola jokic,5.00,34.80,20.40,14.40,10.80,1.80,0.80,1.00,0.59,0.86,3.00,13.43,12.04,2.00
2,"10,004.00",luka doncic,4.00,38.30,41.30,11.50,8.30,1.30,0.30,3.50,0.55,0.80,4.00,11.21,11.40,4.00
3,"10,017.00",lamelo ball,6.00,33.30,23.30,7.80,9.80,1.70,0.20,3.30,0.43,0.81,3.70,7.64,8.47,7.00
4,"10,030.00",chet holmgren,4.00,34.30,23.00,10.30,1.80,1.30,1.30,2.50,0.57,0.86,3.00,7.98,7.84,15.00
5,"10,010.00",james harden,5.00,33.20,21.80,4.80,9.20,1.40,0.20,2.80,0.47,0.91,4.00,7.22,6.23,29.00
6,"10,033.00",josh giddey,6.00,33.70,22.20,9.30,8.70,0.80,0.30,1.80,0.49,0.72,3.30,6.51,5.79,37.00
7,"10,024.00",amen thompson,5.00,33.20,15.00,6.00,5.80,0.80,0.20,0.20,0.47,0.69,2.60,6.70,0.82,184.00



🏀 Starters view (with TOTAL/AVG):


,INDEX,PLAYER_NORM,G,MPG,PTS,REB,AST,STL,BLK,3PM,FG%,FT%,TO,RANK_SCORE_PROJ,RANK_SCORE_LIVE,RANK
0,"10,003.00",victor wembanyama,6.00,118.30,93.45,47.95,11.20,4.55,16.45,3.50,0.53,0.79,8.75,14.92,15.78,1.00
1,"10,001.00",nikola jokic,5.00,121.80,71.40,50.40,37.80,6.30,2.80,3.50,0.59,0.86,10.50,13.43,12.04,2.00
2,"10,004.00",luka doncic,4.00,134.05,144.55,40.25,29.05,4.55,1.05,12.25,0.55,0.80,14.00,11.21,11.40,4.00
3,"10,017.00",lamelo ball,6.00,116.55,81.55,27.30,34.30,5.95,0.70,11.55,0.43,0.81,12.95,7.64,8.47,7.00
4,"10,030.00",chet holmgren,4.00,120.05,80.50,36.05,6.30,4.55,4.55,8.75,0.57,0.86,10.50,7.98,7.84,15.00
5,"10,010.00",james harden,5.00,116.20,76.30,16.80,32.20,4.90,0.70,9.80,0.47,0.91,14.00,7.22,6.23,29.00
6,"10,033.00",josh giddey,6.00,117.95,77.70,32.55,30.45,2.80,1.05,6.30,0.49,0.72,11.55,6.51,5.79,37.00
7,"10,024.00",amen thompson,5.00,116.20,52.50,21.00,20.30,2.80,0.70,0.70,0.47,0.69,9.10,6.70,0.82,184.00
8,,TOTAL / AVG,,961.10,677.95,272.30,201.60,36.40,28.00,56.35,0.51,0.81,91.35,,,


In [7]:
from IPython.display import HTML, display

# 🧮 Cell 6 — Category Counter vs Baselines
baselines = {
    "FG%": 0.478,
    "FT%": 0.796,
    "3PM": 49.38,
    "REB": 178.66,
    "AST": 110.222,
    "STL": 29.121,
    "BLK": 19.034,
    "TO": 58.448,     # lower is better
    "PTS": 503.098,
}

# Get the TOTAL / AVG row
summary_row = starters_summary_df[starters_summary_df["PLAYER_NORM"] == "TOTAL / AVG"]
if summary_row.empty:
    raise ValueError("TOTAL / AVG row not found. Make sure Cell 4 ran successfully.")

summary_row = summary_row.iloc[0]

wins = 0
results = {}

for cat, base in baselines.items():
    if cat not in starters_summary_df.columns:
        results[cat] = "missing"
        continue
    val = pd.to_numeric(summary_row.get(cat), errors="coerce")
    if pd.isna(val):
        results[cat] = "missing"
        continue

    if cat == "TO":
        won = val < base
    else:
        won = val > base

    wins += int(won)
    results[cat] = f"{'✅ WIN' if won else '❌ LOSE'} ({val:,.2f} vs {base:,.2f})"

# Optional table
results_df = pd.DataFrame(
    [(k, baselines[k], results[k]) for k in baselines.keys()],
    columns=["Category", "Baseline", "Result"]
)
display(results_df)

# Fancy HTML banner
html_message = f"""
<div style="
    font-size: 32px; 
    font-weight: bold; 
    color: #2E86C1; 
    margin-top: 25px; 
    border-top: 3px solid #D6DBDF;
    padding-top: 15px;">
    🏀 COLUMBUS BC are projected to win 
    <span style="color: #D35400;">{wins}</span> 
    categories this week! 🏆
</div>
"""

display(HTML(html_message))

,Category,Baseline,Result
0,FG%,0.478,✅ WIN (0.51 vs 0.48)
1,FT%,0.796,✅ WIN (0.81 vs 0.80)
2,3PM,49.380,✅ WIN (56.35 vs 49.38)
3,REB,178.660,✅ WIN (272.30 vs 178.66)
4,AST,110.222,✅ WIN (201.60 vs 110.22)
5,STL,29.121,✅ WIN (36.40 vs 29.12)
6,BLK,19.034,✅ WIN (28.00 vs 19.03)
7,TO,58.448,❌ LOSE (91.35 vs 58.45)
8,PTS,503.098,✅ WIN (677.95 vs 503.10)
